In [ ]:
import pandas as pd

data = pd.read_csv('./transactionsTest.csv').to_dict(orient='records') # Converts csv to pandas dataframe and then to list of dictionaries
print(data)

In [ ]:
groceryTotalAmt = 0
hotelTotalAmt = 0
transportationTotalAmt = 0
restaurantTotalAmt = 0

groceryTotalTransactions = 0
hotelTotalTransactions = 0      
transportationTotalTransactions = 0
restaurantTotalTransactions = 0

groceryDaysDifference = 0
hotelDaysDifference = 0
transportationDaysDifference = 0
restaurantDaysDifference = 0

for transaction in data:
    if transaction['Category'] == 'grocery':
        groceryTotalAmt += transaction['Amount']
        groceryTotalTransactions += 1
    elif transaction['Category'] == 'hotel':
        hotelTotalAmt += transaction['Amount']
        hotelTotalTransactions += 1
    elif transaction['Category'] == 'transportation':
        transportationTotalAmt += transaction['Amount']
        transportationTotalTransactions += 1
    elif transaction['Category'] == 'restaurant':
        restaurantTotalAmt += transaction['Amount']       
        restaurantTotalTransactions += 1

print("Grocery Total: ", groceryTotalAmt)
print("Hotel Total: ", hotelTotalAmt)     
print("Transportation Total: ", transportationTotalAmt)
print("Restaurant Total: ", restaurantTotalAmt)

Grocery Total:  52522.469999999965
Hotel Total:  82078.51000000005
Transportation Total:  16162.680000000008
Restaurant Total:  32038.459999999963
